# Evaluator-Optimizer in DSPy

This tutorial demonstrates how to build and optimize a joke-telling AI using the DSPy framework with the evaluator-optimizer pattern, showing the LLM-as-a-Judge technique and the GEPA evolutionary optimizer.


## What is DSPy?

DSPy (Declarative Self-improving Python) is a framework for programming with language models (LMs) that allows you to:
- Build modular AI programs using composable modules
- Automatically optimize prompts and few-shot examples
- Evaluate and improve your programs systematically

## What We'll Build

In this tutorial, we'll create a joke-telling AI comedian (JokeGPT?) that:
1. Generates jokes about any topic
2. Learns what makes jokes funny through optimization
3. Gets progressively better at telling jokes

## The Evaluator-Optimizer Pattern

In this tutorial, we'll use the evaluator-optimizer pattern, which is a powerful workflow for ensuring our AI program meets all requirements through iterative refinement. Here's how it works:

1. **Generation**: An LLM performs the task (generating jokes in our case)
2. **Evaluation**: A second LLM evaluates if the result meets our criteria (checking if jokes are funny)
3. **Optimization**: The prompt is optimized using GEPA, making adjustments until all requirements are met

This pattern is particularly useful for:
- Ensuring consistent quality in generated content
- Incorporating synthetic feedback to improve outputs
- Systematically optimizing prompts and examples

## Tutorial Structure

1. **Setup**: Install DSPy and configure language models
2. **Basic Programs**: Create simple AI programs with signatures
3. **Modular Programs**: Build reusable components
4. **Evaluation**: Create metrics to measure performance
5. **Optimization**: Automatically improve prompts

In [ ]:
# Install DSPy quietly (-q flag suppresses verbose output)
!pip install dspy python-dotenv pandas -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Import necessary libraries
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file (contains API keys)
load_dotenv()

# Initialize the OpenAI language model
# - "openai/gpt-5-mini" specifies the model to use
# - api_key is loaded from environment variable for security
student_lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), max_tokens=16000, temperature=1)

# Test the language model with a simple query
student_lm("Hello")

['Hi — how can I help you today?']

In [ ]:
# Define custom instructions for our joke generator
instructions = """Tell a funny joke about the topic in the style of the comedian"""

# Define input and output fields with descriptions
fields = {
    # Input field with description
    "topic": (str, dspy.InputField(desc="The topic of the joke")),
    "comedian": (str, dspy.InputField(desc="The comedian to imitate")),

    # Output field with description
    "joke": (str, dspy.OutputField(desc="The joke that is being told")),
}

# Create a signature programmatically
comedian_signature = dspy.make_signature(
    signature_name="Comedian",
    instructions=instructions,
    signature=fields
)

# Create a program with our custom signature
comedian_program = dspy.Predict(comedian_signature)

# Set the LM to use for the program
comedian_program.set_lm(student_lm)

# Test it out
output = comedian_program(topic="AI engineering", comedian="Ricky Gervais")
print(output.joke)

I can't exactly imitate Ricky Gervais, but here's a joke that captures his tone:

Have you noticed how "AI engineering" is just a fancy name for cleaning other people's messes? They hire brilliant people to spend 90% of their time scrubbing spreadsheets called "final_final_v3" and 10% explaining to investors why the model 'hallucinated' that their app is a cat café. Then the CEO comes in, names the thing "Atlas," asks it to "disrupt markets" and treats a 200MB bug report like an existential crisis. It's pathetic — we've trained a computer on the sum of human knowledge and it still can't understand the one thing we all do perfectly: blame someone else.


In [ ]:
# Import random for shuffling
import random
random.seed(69)  # Set seed for reproducibility

# Dataset of professional comedian jokes (labeled as funny)
# Source: Various famous comedians
# https://inews.co.uk/light-relief/jokes/ricky-gervais-jokes-best-golden-globes-2020-host-controversial-funniest-the-office-135797
# https://www.blackpoolgrand.co.uk/funniest-jokes-one-liners/
# https://www.vulture.com/2018/01/dave-chappelle-bird-revelation-equanimity-best-jokes.html
# https://www.scotsman.com/heritage-and-retro/heritage/billy-connollys-best-jokes-80-of-the-big-yins-funniest-jokes-and-one-liners-4458332
# https://inews.co.uk/light-relief/jokes/funny-jokes-110-funniest-best-one-liners-192413

funny_jokes = [
    {"topic": "Fishing", "joke": "Give a man a fish, and he'll probably follow you home expecting more fish.", "comedian": "Ricky Gervais"},
    {"topic": "Family", "joke": "Where there's a will – there's a relative!", "comedian": "Ricky Gervais"},
    {"topic": "Holidays", "joke": "1st of December, World Aids Day….I don't think it'll ever take off like Christmas.", "comedian": "Ricky Gervais"},
    {"topic": "Drinking", "joke": "I like a drink as much as the next man. Unless the next man is Mel Gibson.", "comedian": "Ricky Gervais"},
    {"topic": "Celebrity", "joke": "It's gonna be a night of partying and heavy drinking. Or as Charlie calls it: breakfast.", "comedian": "Ricky Gervais"},
    {"topic": "Movies", "joke": "It seems like everything this year was three-dimensional, except the characters in The Tourist.", "comedian": "Ricky Gervais"},
    {"topic": "Religion", "joke": "You won't burn in hell. But be nice anyway.", "comedian": "Ricky Gervais"},
    {"topic": "Inspiration", "joke": "My greatest hero is Nelson Mandela. What a man. Incarcerated for 25 years, he was released in 1990 and he hasn't reoffended. I think he's going straight, which shows you prison does work.", "comedian": "Ricky Gervais"},
    {"topic": "Philosophy", "joke": "Remember, when you are dead, you do not know you are dead. It is only painful for others. The same applies when you are stupid.", "comedian": "Ricky Gervais"},
    {"topic": "Life", "joke": "Mondays are fine. It's your life that sucks.", "comedian": "Ricky Gervais"},
    {"topic": "Religion", "joke": "Remember, if you don't sin, then Jesus died for nothing.", "comedian": "Ricky Gervais"},
    {"topic": "Activism", "joke": "I could solve the world's problems if I… cared.", "comedian": "Ricky Gervais"},
    {"topic": "Identity", "joke": "I can have a go at the French cause I'm half French half English with a stupid name like Gervais. No I am, I'm half French half English and um I've got qualities of both, French and English which is good, so um… I am crap in bed but at least I've got bad breath.", "comedian": "Ricky Gervais"},
    {"topic": "Military", "joke": "Do commandos not wear pants? They must wear pants, don't they?", "comedian": "Ricky Gervais"},
    {"topic": "Equality", "joke": "Same sex marriage is not a gay privilege, it's equal rights. Privilege would be something like gay people not paying taxes. Like churches don't.", "comedian": "Ricky Gervais"},
    {"topic": "Folklore", "joke": "I've never worked out what the moral of Humpty Dumpty is. I can only think of: Don't sit on a wall, if you're an egg.", "comedian": "Ricky Gervais"},
    {"topic": "Employment", "joke": "Avoid employing unlucky people – throw half of the pile of CVs in the bin without reading them.", "comedian": "Ricky Gervais"},
    {"topic": "Awards", "joke": "For any of you who don't know, the Golden Globes are just like the Oscars, but without all that esteem. The Golden Globes are to the Oscars what Kim Kardashian is to Kate Middleton. A bit louder, a bit trashier, a bit drunker, and more easily bought.", "comedian": "Ricky Gervais"},
    {"topic": "Workplace", "joke": "If your boss is getting you down, look at him through the prongs of a fork and imagine him in jail.", "comedian": "Ricky Gervais"},
    {"topic": "Humor", "joke": "I can't find someone funny whom I don't like. Hitler told great jokes.", "comedian": "Ricky Gervais"},
    {"topic": "Culture", "joke": "America champions the underdog. We champion the under dog until he's not the underdog anymore, and he annoys us.", "comedian": "Ricky Gervais"},
    {"topic": "Betrayal", "joke": "You have to be 100% behind someone, before you can stab them in the back.", "comedian": "Ricky Gervais"},
    {"topic": "Health", "joke": "Remember, being healthy is basically dying as slowly as possible.", "comedian": "Ricky Gervais"},
    {"topic": "Atheism", "joke": "I'd like to thank God for making me an atheist.", "comedian": "Ricky Gervais"},
    {"topic": "Music Industry", "joke": "Piracy doesn't kill music, boy bands do.", "comedian": "Ricky Gervais"},
    {"topic": "Wealth", "joke": "My wealth and happiness would suggest that God definitely does love me. If he existed of course. Which he doesn't.", "comedian": "Ricky Gervais"},
    {"topic": "Social Media", "joke": "Following someone on Twitter and asking them to tweet about something else is like stalking someone and asking them to go a different route.", "comedian": "Ricky Gervais"},
    {"topic": "Fame", "joke": "Please don't worship me. I'm just an ordinary guy, with lots of followers trying to spread my message. Sort of like Jesus Christ I guess.", "comedian": "Ricky Gervais"},
    {"topic": "Technology", "joke": "iPhones are Barbie Dolls for grown men. You carry them round, dress them up in little outfits, accessorise, & get a new one every year.", "comedian": "Ricky Gervais"},
    {"topic": "Generosity", "joke": "Give a man a fish, and he'll probably follow you home expecting more fish.", "comedian": "Ricky Gervais"},
    {"topic": "Environment", "joke": "It seems to be true, particularly in middle America, that those most militant about using up fossil fuels, don't actually believe in fossils", "comedian": "Ricky Gervais"},
    {"topic": "Drinking", "joke": "My father drank so heavily, when he blew on the birthday cake he lit the candles.", "comedian": "Les Dawson"},
    {"topic": "Police", "joke": "I was in my car driving back from work. A police officer pulled me over and knocked on my window. I said, 'One minute I'm on the phone.'", "comedian": "Alan Carr"},
    {"topic": "Overthinking", "joke": "I worry about ridiculous things, you know, how does a guy who drives a snowplough get to work in the morning… that can keep me awake for days.", "comedian": "Billy Connolly"},
    {"topic": "Relationships", "joke": "I used to go out with a giraffe. Used to take it to the pictures and that. You'd always get some bloke complaining that he couldn't see the screen.", "comedian": "Paul Merton"},
    {"topic": "Music", "joke": "Here's a picture of me with REM. That's me in the corner.", "comedian": "Milton Jones"},
    {"topic": "Optimism", "joke": "People say 'Bill, are you an optimist?' And I say, 'I hope so.'", "comedian": "Bill Bailey"},
    {"topic": "Customer Service", "joke": "I rang up British Telecom and said: 'I want to report a nuisance caller.' He said: 'Not you again.'", "comedian": "Tim Vine"},
    {"topic": "Obesity", "joke": "Life is like a box of chocolates. It doesn't last long if you're fat.", "comedian": "Joe Lycett"},
    {"topic": "Religion", "joke": "We weren't very religious. On Hanukkah, my mother had our menorah on a dimmer.", "comedian": "Richard Lewis"},
    {"topic": "Beauty", "joke": "My girlfriend is absolutely beautiful. Body like a Greek statue – completely pale, no arms.", "comedian": "Phil Wang"},
    {"topic": "Weather", "joke": "Normally you have news, weather and travel. But not on snow day. On a snow day, the news is weather is travel.", "comedian": "Michael McIntyre"},
    {"topic": "Personal Improvement", "joke": "I bought myself some glasses. My observational comedy improved.", "comedian": "Sara Pascoe"},
    {"topic": "Sports", "joke": "If I was an Olympic athlete, I'd rather come in last than win the silver medal. You win the gold, you feel good. You win the bronze, you think, 'at least I got something.' But you win that silver, that's like, 'Congratulations, you almost won! Of all the losers, you came in first! You're the number one loser! No one lost ahead of you!'", "comedian": "Jerry Seinfeld"},
    {"topic": "Identity", "joke": "My star sign is Pyrex. I was a test-tube baby.", "comedian": "Billy Connolly"},
    {"topic": "Marriage", "joke": "I always take my wife morning tea in my pyjamas. But is she grateful? No, she says she'd rather have it in a cup.", "comedian": "Eric Morecambe"},
    {"topic": "Shopping", "joke": "A man walks into a chemist's and says, 'Can I have a bar of soap, please?' The chemist says, 'Do you want it scented?' And the man says, 'No, I'll take it with me now.'", "comedian": "Ronnie Barker"},
    {"topic": "Crime", "joke": "Crime in multi-storey car parks. That is wrong on so many different levels.", "comedian": "Tim Vine"},
    {"topic": "Social Class", "joke": "You know you're working class when your TV is bigger than your bookcase.", "comedian": "Rob Beckett"},
    {"topic": "Animals", "joke": "Owls haven't got necks, have they? An owl is essentially a one-piece unit.", "comedian": "Ross Noble"},
    {"topic": "Fashion", "joke": "If you arrive fashionably late in Crocs, you're just late.", "comedian": "Joel Dommett"},
    {"topic": "Technology", "joke": "My phone will ring at 2am and my wife'll look at me and go, \"Who's that calling at this time?\" I say, \"I don't know. If I knew that we wouldn't need the bloody phone.\"", "comedian": "Lee Evans"},
    {"topic": "Philosophy", "joke": "I doubt there's a heaven; I think the people from hell have probably bought it for a timeshare.", "comedian": "Victoria Wood"},
    {"topic": "Fitness", "joke": "I said to the gym instructor: \"Can you teach me to do the splits?\", He said: \"How flexible are you?\", I said: \"I can't make Tuesdays.\"", "comedian": "Tommy Cooper"},
    {"topic": "Insurance", "joke": "Do Transformers get car, or life insurance?", "comedian": "Russell Howard"},
    {"topic": "Police", "joke": "Alright lads, a giant fly is attacking the police station. I've called the SWAT team!", "comedian": "Greg Davies"},
    {"topic": "Healthcare", "joke": "A good rule to remember for life is that when it comes to plastic surgery and sushi, never be attracted by a bargain.", "comedian": "Graham Norton"},
    {"topic": "Animals", "joke": "Two monkeys were getting into the bath. One said: 'Oo, oo, oo, aah aah aah.' The other replied: 'Well, put some cold in it then.'", "comedian": "Harry Hill"},
    {"topic": "Suburban Life", "joke": "My parents did just well enough so I could grow up poor around white people. When Nas and them used to talk about the projects, I used to get jealous. It sounded fun. Everybody in the projects was poor, and that's fair. But if you were poor in Silver Spring, n****, it felt like it was only happening to you.", "comedian": "Dave Chappelle"},
    {"topic": "Cultural Identity", "joke": "What is Rachel willing to do, so that we blacks believe that she believes she is actually one of us? Bitch, are you willing to put a lien on your house so that you can invest in a mixtape that probably won't work out?", "comedian": "Dave Chappelle"},
    {"topic": "Aging", "joke": "I don't like looking at my dick anymore. My dick looks distinguished. It's old, an old-looking dick. It's got salt-and-pepper hair all around it. My dick looks like Morgan Freeman in the '90s.", "comedian": "Dave Chappelle"},
    {"topic": "Fatherhood", "joke": "This motherfucker calls me up in the middle of the night. It was one o'clock in the morning and he goes, 'Dad, don't be mad […] I'm at a party and my designated driver had too much to drink. Me and friends need you to come pick us up.' I said, 'Jesus Christ, it's one o'clock in the morning. N****, I am shit-faced!'", "comedian": "Dave Chappelle"},
    {"topic": "Political Commentary", "joke": "Eight years later, I'm pulling up to the polls again. This time, I'm driving a brand-new Porsche because the Obama years were very good to me […] I walked up and saw a long, long line of dusty white people […] I stood with them in line, like all us Americans are required to do in a democracy. Nobody skips the line to vote. And I listened to them say naïve, poor white people things.", "comedian": "Dave Chappelle"},
    {"topic": "Leadership", "joke": "This motherfucker [Donald Trump] grabbed the podium and he goes, 'You don't know how scary the things I read in my briefings are.' Holy shit, man, you ain't supposed to tell us that, bro!", "comedian": "Dave Chappelle"},
    {"topic": "Religious Satire", "joke": "I respect everybody's beliefs, except Amish people. They are the only ones I can say clearly, 'Their God is wrong.' The speed limit is 75 miles an hour in Ohio, and one lane of traffic is blocked by a goddamned horse and buggy?", "comedian": "Dave Chappelle"},
    {"topic": "Hollywood", "joke": "You think I go to a Hollywood meeting with all them white people by myself? I bring my N**** Mac Mittens from the streets […] He's not even qualified to listen to these meetings, he just makes me feel good.", "comedian": "Dave Chappelle"},
    {"topic": "Comedy Culture", "joke": "The tough part of being a comedian and knowing the motherfucker is, everybody comes up to me like, 'Did you know? Did you know what Louis was doing?' No, bitch, I did not know.", "comedian": "Dave Chappelle"},
    {"topic": "National Identity", "joke": "I could kill every white person in America at one time. You know how I'd do it? Just wait for the Super Bowl, and right when they sing the National Anthem, I'd have O.J. Simpson walk to the 50-yard line with them bad knees.", "comedian": "Dave Chappelle"},
    {"topic": "Gender Relations", "joke": "I used to do shows for drug dealers that wanted to clean their money up. One time I did a real good set, and these motherfuckers called me into the back room. They gave me $25,000 in cash […] I jumped on the subway and started heading towards Brooklyn at one o'clock in the morning.", "comedian": "Dave Chappelle"},
    {"topic": "Scottish Heritage", "joke": "Scottish-Americans tell you that if you want to identify tartans, it's easy – you simply look under the kilt, and if it's a quarter-pounder, you know it's a McDonald's.", "comedian": "Billy Connolly"},
    {"topic": "Judgement", "joke": "Before you judge a man, walk a mile in his shoes. After that who cares? He's a mile away and you've got his shoes!", "comedian": "Billy Connolly"},
    {"topic": "Weather", "joke": "I hate all those weathermen, too, who tell you that rain is bad weather. There's no such thing as bad weather, just the wrong clothing, so get yourself a sexy raincoat and live a little.", "comedian": "Billy Connolly"},
    {"topic": "Film Industry", "joke": "I'm a huge film star, but you have to hurry to the movies because I usually die in the first 15 f***ing minutes. I'm the only guy I know who died in a f***ing Muppet Movie.", "comedian": "Billy Connolly"},
    {"topic": "Appearance", "joke": "I always look skint. When I buy a Big Issue, people take it out of my hand and give me a pound.", "comedian": "Billy Connolly"},
    {"topic": "Sex Therapy", "joke": "One sex therapist claims that the most effective way to arouse your man is to spend 10 minutes licking his ears. Personally, I think its bollocks.", "comedian": "Billy Connolly"},
    {"topic": "Cinema", "joke": "When people say while watching a film 'did you see that? No tosser, I paid ten quid to come to the cinema and stare at the f***ing floor.", "comedian": "Billy Connolly"},
    {"topic": "Aeroplane Comfort", "joke": "I get claustrophobic easily and I don't get why aeroplane toilets don't f***ing have windows. I mean it's not as if anyone can f***ing see in. Unless of course you are the most determined pervert in the world.", "comedian": "Billy Connolly"},
    {"topic": "Astrology", "joke": "My star sign is Pyrex. I was a test-tube baby.", "comedian": "Billy Connolly"},
    {"topic": "Parenting", "joke": "Don't buy one of those baby intercoms. Babies pretend to be dead. They're bastards, and they do it on purpose.", "comedian": "Billy Connolly"},
    {"topic": "Common Sayings", "joke": "Why do people say 'Oh you want to have your cake and eat it too?' Dead right! What good is a cake if you can't eat it?", "comedian": "Billy Connolly"},
    {"topic": "Life Perception", "joke": "When people say 'life is short'. What the f***? Life is the longest damn thing anyone ever f***ing does! What can you do that's longer?", "comedian": "Billy Connolly"},
    {"topic": "Dating", "joke": "I like a woman with a head on her shoulders. I hate necks.", "comedian": "Steve Martin"},
    {"topic": "Growing Up", "joke": "I have a lot of growing up to do. I realised that the other day inside my fort.", "comedian": "Zach Galifianakis"},
    {"topic": "Employment", "joke": "I used to work at McDonald's making minimum wage. You know what that means when someone pays you minimum wage? You know what your boss was trying to say? 'Hey, if I could pay you less, I would, but it's against the law.'", "comedian": "Chris Rock"},
    {"topic": "Love", "joke": "Love is like a fart. If you have to force it it's probably s***.", "comedian": "Stephen K. Amos"},
    {"topic": "Convenience", "joke": "I like an escalator because an escalator can never break. It can only become stairs. There would never be an 'Escalator Temporarily Out of Order' sign, only 'Escalator Temporarily Stairs'.", "comedian": "Mitch Hedberg"},
    {"topic": "Sports", "joke": "If I was an Olympic athlete, I'd rather come in last than win the silver medal. You win the gold, you feel good. You win the bronze, you think, 'at least I got something.' But you win that silver, that's like, 'Congratulations, you almost won! Of all the losers, you came in first! You're the number one loser! No one lost ahead of you!'", "comedian": "Jerry Seinfeld"},
    {"topic": "Religion", "joke": "We weren't very religious. On Hanukkah, my mother had our menorah on a dimmer.", "comedian": "Richard Lewis"},
    {"topic": "Beauty", "joke": "My girlfriend is absolutely beautiful. Body like a Greek statue – completely pale, no arms.", "comedian": "Phil Wang"},
    {"topic": "Creation", "joke": "If God had written the Bible, the first line should have been 'It's round.'", "comedian": "Eddie Izzard"},
    {"topic": "Self-Improvement", "joke": "I bought myself some glasses. My observational comedy improved.", "comedian": "Sara Pascoe"},
    {"topic": "Politics", "joke": "Trump's nothing like Hitler. There's no way he could write a book.", "comedian": "Frankie Boyle"},
    {"topic": "Social Class", "joke": "You know you're working class when your TV is bigger than your book case.", "comedian": "Rob Beckett"},
    {"topic": "Conflict", "joke": "Most of my life is spent avoiding conflict. I hardly ever visit Syria.", "comedian": "Alex Horne"},
    {"topic": "Relaxation", "joke": "A spa hotel? It's like a normal hotel, only in reception there's a picture of a pebble.", "comedian": "Rhod Gilbert"},
    {"topic": "Health", "joke": "Life is like a box of chocolates. It doesn't last long if you're fat.", "comedian": "Joe Lycett"},
    {"topic": "Career", "joke": "My Dad said, always leave them wanting more. Ironically, that's how he lost his job in disaster relief.", "comedian": "Mark Watson"},
    {"topic": "Memory", "joke": "Apparently smoking cannabis can affect your short term memory. Well if that's true, what do you think smoking cannabis does?", "comedian": "Mickey P Kerr"},
    {"topic": "Philosophy", "joke": "How many philosophers does it take to change a lightbulb?…. none. They're not really into that sort of thing. If it's that dark, light a candle.", "comedian": "Phil Cornwell"},
    {"topic": "Marriage", "joke": "The first time I met my wife, I knew she was a keeper. She was wearing massive gloves.", "comedian": "Alun Cochrane"},
    {"topic": "Childhood", "joke": "As a kid I was made to walk the plank. We couldn't afford a dog.", "comedian": "Gary Delaney"},
    {"topic": "Misunderstanding", "joke": "Two fish in a tank. One says: 'How do you drive this thing?'", "comedian": "Peter Kay"},
    {"topic": "Entertainment", "joke": "I saw a documentary on how ships are kept together. Riveting!", "comedian": "Stewart Francis"},
    {"topic": "Music", "joke": "People who like trance music are very persistent. They don't techno for an answer.", "comedian": "Joel Dommett"},
    {"topic": "Dating", "joke": "I used to go out with a giraffe. Used to take it to the pictures and that. You'd always get some bloke complaining that he couldn't see the screen. It's a giraffe, mate. What do you expect? 'Well he can take his hat off for a start!'", "comedian": "Paul Merton"},
    {"topic": "Weather", "joke": "Normally you have news, weather and travel. But not on snow day. On a snow day, news is weather is travel.", "comedian": "Michael McIntyre"},
    {"topic": "Music", "joke": "Here's a picture of me with REM. That's me in the corner.", "comedian": "Milton Jones"},
    {"topic": "Sarcasm", "joke": "Someone showed me a photograph of my local MP the other day. 'Would you buy a second-hand car from this man?' they asked. 'Would you buy a second-hand car?' I replied.", "comedian": "Miles Jupp"},
    {"topic": "Culture", "joke": "With stand-up in Britain, what you have to do is bloody swearing. In Germany, we don't have to swear. Reason being, things work.", "comedian": "Henning When"},
    {"topic": "Learning", "joke": "I'm learning the hokey cokey. Not all of it. But – I've got the ins and outs.", "comedian": "Iain Stirling"},
    {"topic": "Identity", "joke": "Roses are red, violets are blue, I'm a schizophrenic, and so am I.", "comedian": "Billy Connolly"},
    {"topic": "Parenting", "joke": "My mother told me, you don't have to put anything in your mouth you don't want to. Then she made me eat broccoli, which felt like double standards.", "comedian": "Sarah Millican"},
    {"topic": "Vengeance", "joke": "My therapist says I have a preoccupation with vengeance. We'll see about that.", "comedian": "Stewart Francis"},
    {"topic": "Family", "joke": "I'm sure wherever my Dad is, he's looking down on us. He's not dead, just very condescending.", "comedian": "Jack Whitehall"},
    {"topic": "Marriage", "joke": "'What's a couple?' I asked my mum. She said, 'Two or three'. Which probably explains why her marriage collapsed.", "comedian": "Josie Long"},
    {"topic": "Injury", "joke": "The easiest time to add insult to injury is when you're signing somebody's cast.", "comedian": "Demetri Martin"},
    {"topic": "Communication", "joke": "I was in my car driving back from work. A police officer pulled me over and knocked on my window. I said, 'One minute I'm on the phone.'", "comedian": "Alan Carr"},
    {"topic": "Afterlife", "joke": "I doubt there's a heaven; I think the people from hell have probably bought it for a timeshare.", "comedian": "Victoria Wood"},
    {"topic": "Flexibility", "joke": "I said to the gym instructor: 'Can you teach me to do the splits?' He said: 'How flexible are you?' I said: 'I can't make Tuesdays.'", "comedian": "Tommy Cooper"},
    {"topic": "Misunderstanding", "joke": "A man walks into a chemist's and says, 'Can I have a bar of soap, please?' The chemist says, 'Do you want it scented?' And the man says, 'No, I'll take it with me now.'", "comedian": "Ronnie Barker"},
    {"topic": "Humor", "joke": "It's really hard to define 'virtue signalling', as I was saying the other day to some of my Muslim friends over a fair-trade coffee in our local feminist bookshop.", "comedian": "Lucy Porter"},
    {"topic": "Creation", "joke": "If we were truly created by God, then why do we still occasionally bite the insides of our own mouths?", "comedian": "Dara Ó Briain"},
    {"topic": "Insurance", "joke": "Do Transformers get car, or life insurance?", "comedian": "Russell Howard"},
    {"topic": "Emergency", "joke": "Alright lads, a giant fly is attacking the police station. I've called the SWAT team!", "comedian": "Greg Davies"},
    {"topic": "Consumerism", "joke": "A good rule to remember for life is that when it comes to plastic surgery and sushi, never be attracted by a bargain.", "comedian": "Graham Norton"},
    {"topic": "Family", "joke": "My father drank so heavily, when he blew on the birthday cake he lit the candles.", "comedian": "Les Dawson"},
    {"topic": "Therapy", "joke": "I've been feeling suicidal so my therapist suggested I do CBT. Now I can ride a motorbike, how's that going to help?", "comedian": "Eric Lampaert"},
]

# Dataset of generic, unfunny jokes (labeled as not funny) - generated by ChatGPT
unfunny_jokes = [
    {"topic": "Science", "joke": "Why don't scientists trust atoms? Because they make up everything."},
    {"topic": "Field", "joke": "Why did the scarecrow win an award? Because he was outstanding in his field."},
    {"topic": "Animals", "joke": "Why do cows have hooves instead of feet? Because they lactose."},
    {"topic": "Food", "joke": "What do you call fake spaghetti? An impasta."},
    {"topic": "Animals", "joke": "How does a penguin build its house? Igloos it together."},
    {"topic": "Halloween", "joke": "What do you get when you cross a snowman and a vampire? Frostbite."},
    {"topic": "Books", "joke": "Why was the math book sad? It had too many problems."},
    {"topic": "Food", "joke": "What do you call cheese that isn't yours? Nacho cheese."},
    {"topic": "Skeletons", "joke": "Why don't skeletons fight each other? They don't have the guts."},
    {"topic": "Walls", "joke": "What did one wall say to the other wall? I'll meet you at the corner."},
    {"topic": "Transportation", "joke": "Why did the bicycle fall over? It was two-tired."},
    {"topic": "Animals", "joke": "What do you call a bear with no teeth? A gummy bear."},
    {"topic": "Gym", "joke": "Why don't some couples go to the gym? Because some relationships don't work out."},
    {"topic": "Factories", "joke": "What do you call a factory that makes good products? A satisfactory."},
    {"topic": "Golf", "joke": "Why did the golfer bring an extra pair of pants? In case he got a hole in one."},
    {"topic": "Cleaning", "joke": "What did the janitor say when he jumped out of the closet? Supplies!"},
    {"topic": "Animals", "joke": "What do you call a fish with no eyes? Fsh."},
    {"topic": "Charity", "joke": "Why don't oysters donate to charity? Because they are shellfish."},
    {"topic": "Food", "joke": "What did the grape do when it got stepped on? Nothing but let out a little wine."},
    {"topic": "Animals", "joke": "Why was the big cat disqualified from the race? Because it was a cheetah."},
    {"topic": "Fashion", "joke": "What do you call a belt made of watches? A waist of time."},
    {"topic": "Body", "joke": "Why can't your nose be 12 inches long? Because then it would be a foot."},
    {"topic": "Sports", "joke": "Why don't some fish play basketball? Because they are afraid of the net."},
    {"topic": "Animals", "joke": "What do you call a pile of cats? A meowtain."},
    {"topic": "Coffee", "joke": "Why did the coffee file a police report? It got mugged."},
    {"topic": "Weather", "joke": "Why did the stadium get hot after the game? All the fans left."},
    {"topic": "Plates", "joke": "What did one plate say to the other plate? Lunch is on me."},
    {"topic": "Space", "joke": "How do you organize a space party? You planet."},
    {"topic": "Food", "joke": "Why don't eggs tell jokes? They'd crack each other up."},
    {"topic": "Halloween", "joke": "How does a vampire start a letter? Tomb it may concern."},
    {"topic": "Technology", "joke": "Why did the computer go to the doctor? It had a virus."},
    {"topic": "Boomerangs", "joke": "What do you call a boomerang that doesn't come back? A stick."},
    {"topic": "Ghosts", "joke": "Why are ghosts bad at lying? Because you can see right through them."},
    {"topic": "Animals", "joke": "What do you get when you cross a sheep and a kangaroo? A woolly jumper."},
    {"topic": "Food", "joke": "Why did the tomato turn red? Because it saw the salad dressing."},
    {"topic": "School", "joke": "Why did the math teacher take off points? Because the student's answer was too square."},
    {"topic": "Birds", "joke": "Why do seagulls fly over the ocean? Because if they flew over the bay, they'd be bagels."},
    {"topic": "Food", "joke": "Why was the baby strawberry crying? Because its parents were in a jam."},
    {"topic": "Technology", "joke": "What do you call a droid that takes the long way around? R2 detour."},
    {"topic": "Fashion", "joke": "Why did the scarecrow get promoted? He was outstanding in his field."},
    {"topic": "Fashion", "joke": "What did one hat say to the other hat? You stay here, I'll go on ahead."},
    {"topic": "Fashion", "joke": "Why was the belt arrested? It held up a pair of pants."},
    {"topic": "Animals", "joke": "What do you call an alligator in a vest? An investigator."},
    {"topic": "Animals", "joke": "Why don't you see elephants hiding in trees? Because they're so good at it."},
    {"topic": "Books", "joke": "Why did the math book look sad? Because it had too many problems."},
    {"topic": "Bees", "joke": "Why do bees have sticky hair? Because they use honeycombs."},
    {"topic": "Music", "joke": "Why did the chicken join a band? Because it had the drumsticks."},
    {"topic": "Animals", "joke": "How do you catch a squirrel? Climb a tree and act like a nut."},
    {"topic": "Technology", "joke": "Why was the computer cold? It left its Windows open."},
    {"topic": "Animals", "joke": "What do you call a magic dog? A labracadabrador."},
    {"topic": "Sports", "joke": "Why don't some fish play basketball? Because they're afraid of the net."},
    {"topic": "Oceans", "joke": "What did one ocean say to the other ocean? Nothing, they just waved."},
    {"topic": "Dogs", "joke": "Why did the cowboy get a dachshund? Because he wanted to get a long little doggie."},
    {"topic": "Snowmen", "joke": "What do you call a snowman with a six-pack? An abdominal snowman."},
    {"topic": "Food", "joke": "Why did the tomato turn red? Because it saw the salad dressing."},
    {"topic": "Animals", "joke": "How does a penguin build its house? Igloos it together."},
    {"topic": "Golf", "joke": "Why did the golfer bring extra pants? In case he got a hole in one."},
    {"topic": "Animals", "joke": "What do you call an alligator in a vest? An investigator."},
    {"topic": "Fashion", "joke": "Why do cows wear bells? Because their horns don't work."},
    {"topic": "Field", "joke": "Why did the scarecrow become a successful neurosurgeon? Because he was outstanding in his field."},
    {"topic": "Cleaning", "joke": "What did the janitor say when he jumped out of the closet? Supplies!"},
    {"topic": "Science", "joke": "Why don't scientists trust atoms? Because they make up everything."},
    {"topic": "Skeletons", "joke": "Why did the skeleton go to the party alone? He had no body to go with him."},
    {"topic": "Transportation", "joke": "Why did the bicycle fall over? It was two-tired."},
    {"topic": "Technology", "joke": "Why did the computer go to the doctor? It had a virus."},
    {"topic": "Food", "joke": "What did the grape do when it got stepped on? Nothing but let out a little wine."},
    {"topic": "Ghosts", "joke": "Why do ghosts like elevators? Because it lifts their spirits."},
    {"topic": "Science", "joke": "Why can't you trust an atom? Because they make up everything."},
    {"topic": "Food", "joke": "What do you call fake spaghetti? An impasta."},
    {"topic": "Cleaning", "joke": "How do you make a tissue dance? Put a little boogie in it."},
    {"topic": "Charity", "joke": "Why don't oysters donate to charity? Because they are shellfish."},
    {"topic": "Boomerangs", "joke": "What do you call a boomerang that doesn't come back? A stick."},
    {"topic": "Books", "joke": "Why did the math book look sad? Because it had too many problems."},
    {"topic": "Skeletons", "joke": "Why don't skeletons fight each other? They don't have the guts."},
    {"topic": "Walls", "joke": "What did one wall say to the other wall? I'll meet you at the corner."},
    {"topic": "Animals", "joke": "What do you call a bear with no teeth? A gummy bear."},
    {"topic": "Plates", "joke": "What did one plate say to the other plate? Lunch is on me."},
    {"topic": "Space", "joke": "How do you organize a space party? You planet."},
    {"topic": "Food", "joke": "Why don't eggs tell jokes? They'd crack each other up."},
    {"topic": "Halloween", "joke": "How does a vampire start a letter? Tomb it may concern."},
    {"topic": "Coffee", "joke": "Why did the coffee file a police report? It got mugged."},
    {"topic": "Golf", "joke": "Why did the golfer bring an extra pair of pants? In case he got a hole in one."},
    {"topic": "Animals", "joke": "What do you call a fish with no eyes? Fsh."},
    {"topic": "Food", "joke": "Why did the tomato turn red? Because it saw the salad dressing."},
    {"topic": "Birds", "joke": "Why don't seagulls fly over the bay? Because then they'd be bagels."},
    {"topic": "Food", "joke": "Why do cows have hooves instead of feet? Because they lactose."},
    {"topic": "Sports", "joke": "Why don't some fish play basketball? Because they're afraid of the net."},
    {"topic": "Field", "joke": "Why did the scarecrow win an award? Because he was outstanding in his field."},
    {"topic": "Food", "joke": "What do you call cheese that isn't yours? Nacho cheese."},
    {"topic": "Transportation", "joke": "Why did the bicycle fall over? It was two-tired."},
    {"topic": "Animals", "joke": "How does a penguin build its house? Igloos it together."},
    {"topic": "Animals", "joke": "What do you call a pile of cats? A meowtain."},
    {"topic": "Fashion", "joke": "What did one hat say to the other hat? You stay here, I'll go on ahead."},
    {"topic": "Animals", "joke": "What do you call an alligator in a vest? An investigator."},
    {"topic": "Charity", "joke": "Why don't oysters donate to charity? Because they are shellfish."},
    {"topic": "Food", "joke": "What did the grape do when it got stepped on? Nothing but let out a little wine."},
    {"topic": "Golf", "joke": "Why did the golfer bring an extra pair of pants? In case he got a hole in one."},
    {"topic": "Food", "joke": "Why was the baby strawberry crying? Because its parents were in a jam."},
    {"topic": "Factories", "joke": "What do you call a factory that makes good products? A satisfactory."},
    {"topic": "Skeletons", "joke": "Why don't skeletons fight each other? They don't have the guts."},
    {"topic": "Animals", "joke": "What do you call a fish with no eyes? Fsh."},
    {"topic": "Gym", "joke": "Why don't some couples go to the gym? Because some relationships don't work out."},
    {"topic": "Field", "joke": "Why did the scarecrow win an award? Because he was outstanding in his field."},
    {"topic": "Food", "joke": "What do you call fake spaghetti? An impasta."},
    {"topic": "Halloween", "joke": "How does a vampire start a letter? Tomb it may concern."},
    {"topic": "Technology", "joke": "Why did the computer go to the doctor? It had a virus."},
    {"topic": "Boomerangs", "joke": "What do you call a boomerang that doesn't come back? A stick."},
    {"topic": "Food", "joke": "Why did the tomato turn red? Because it saw the salad dressing."},
    {"topic": "Birds", "joke": "Why do seagulls fly over the ocean? Because if they flew over the bay, they'd be bagels."},
    {"topic": "Food", "joke": "Why was the baby strawberry crying? Because its parents were in a jam."},
    {"topic": "Technology", "joke": "What do you call a droid that takes the long way around? R2 detour."},
    {"topic": "Fashion", "joke": "Why did the scarecrow get promoted? He was outstanding in his field."},
    {"topic": "Fashion", "joke": "What did one hat say to the other hat? You stay here, I'll go on ahead."},
    {"topic": "Fashion", "joke": "Why was the belt arrested? It held up a pair of pants."},
    {"topic": "Animals", "joke": "What do you call an alligator in a vest? An investigator."},
    {"topic": "Animals", "joke": "Why don't you see elephants hiding in trees? Because they're so good at it."},
    {"topic": "Books", "joke": "Why did the math book look sad? Because it had too many problems."},
    {"topic": "Bees", "joke": "Why do bees have sticky hair? Because they use honeycombs."},
    {"topic": "Music", "joke": "Why did the chicken join a band? Because it had the drumsticks."},
    {"topic": "Animals", "joke": "How do you catch a squirrel? Climb a tree and act like a nut."},
    {"topic": "Technology", "joke": "Why was the computer cold? It left its Windows open."},
    {"topic": "Animals", "joke": "What do you call a magic dog? A labracadabrador."},
    {"topic": "Sports", "joke": "Why don't some fish play basketball? Because they're afraid of the net."},
    {"topic": "Oceans", "joke": "What did one ocean say to the other ocean? Nothing, they just waved."},
    {"topic": "Dogs", "joke": "Why did the cowboy get a dachshund? Because he wanted to get a long little doggie."},
    {"topic": "Snowmen", "joke": "What do you call a snowman with a six-pack? An abdominal snowman."},
    {"topic": "Food", "joke": "Why did the tomato turn red? Because it saw the salad dressing."}
]

# Convert to DSPy format
dataset = []

# Process funny jokes
for row in funny_jokes:
    topic, joke, comedian = row["topic"], row["joke"], row["comedian"]

    # Create DSPy Example with labels
    dataset.append(dspy.Example(topic=topic, comedian=comedian, joke=joke, funny=True).with_inputs("topic", "comedian", "joke"))

# Process unfunny jokes
for row in unfunny_jokes:
    topic, joke = row["topic"], row["joke"]
    dataset.append(dspy.Example(topic=topic, joke=joke, comedian=None, funny=False).with_inputs("topic", "comedian", "joke"))

# Shuffle the dataset
random.shuffle(dataset)

# Split into 60% training, 20% validation, 20% development
num_items = len(dataset)
train_index = int(0.6 * num_items)
val_index = int(0.8 * num_items)

trainset = dataset[:train_index]
valset = dataset[train_index:val_index]
devset = dataset[val_index:]

print(f"Training set size: {len(trainset)}")
print(f"Validation set size: {len(valset)}")
print(f"Development set size: {len(devset)}")

trainset[0]

Training set size: 152
Validation set size: 51
Development set size: 51


Example({'topic': 'Books', 'joke': 'Why did the math book look sad? Because it had too many problems.', 'comedian': None, 'funny': False}) (input_keys={'joke', 'topic', 'comedian'})

In [ ]:
# Create a joke judge with Chain of Thought reasoning
# Input: topic and joke, Output: funny (boolean)

# Define custom instructions for our joke judge
instructions = """You are in the audience at a comedy show and must decide if the joke is funny."""

# Define input and output fields with descriptions
fields = {
    # Input field with description
    "topic": (str, dspy.InputField(desc="The topic of the joke")),
    "joke": (str, dspy.InputField(desc="The joke that is being told")),

    # Output field with description
    "funny": (bool, dspy.OutputField(desc="Whether the joke is funny")),
}

# Create a signature programmatically
audience_signature = dspy.make_signature(
    signature_name="Audience",
    instructions=instructions,
    signature=fields
)

audience_program = dspy.ChainOfThought(audience_signature)
audience_program.set_lm(student_lm)

# Test on our first training example
result = audience_program(topic=trainset[0].topic, joke=trainset[0].joke)
print(f"Joke: {trainset[0].joke}")
print(f"\nJudge: {result.reasoning}")
print(f"\nPred: {result.funny}")
print(f"Gold: {trainset[0].funny}")

Joke: Why did the math book look sad? Because it had too many problems.

Judge: This is a classic, family-friendly pun that plays on the double meaning of "problems" — math exercises and personal troubles. It's simple, predictable, and likely to get a harmless chuckle or groan rather than a big laugh, but it's effective for light, inoffensive humor.

Pred: True
Gold: False


In [ ]:
# Define our evaluation metric
def exact_match(gold: dspy.Example, pred: dspy.Prediction, trace=None,pred_name=None, pred_trace=None):
    """Check if the predicted 'funny' label matches the gold answer"""
    return gold.funny == pred.funny

# Create an evaluator
evaluate_audience = dspy.Evaluate(
    metric=exact_match,
    devset=devset,  # the optimized judge hasn't seen this data yet
    num_threads=16, # Run evaluations in parallel
    display_progress=True,
    display_table=10   # Show first 10 results
)

# Evaluate our basic judge
baseline_judge_score = evaluate_audience(audience_program)
print(f"\nBasic judge accuracy: {baseline_judge_score}%")

Average Metric: 24.00 / 51 (47.1%): 100%|██████████| 51/51 [00:00<00:00, 2950.88it/s]

2025/09/25 12:36:13 INFO dspy.evaluate.evaluate: Average Metric: 24 / 51 (47.1%)


,topic,joke,comedian,example_funny,reasoning,pred_funny,exact_match
0,Field,Why did the scarecrow become a successful neurosurgeon? Because he...,None,False,"This is a classic pun that hinges on the double meaning of ""outsta...",True,
1,Ghosts,Why are ghosts bad at lying? Because you can see right through them.,None,False,"This is a simple, clean pun that hinges on the double meaning of ""...",True,
2,Marriage,"The first time I met my wife, I knew she was a keeper. She was wea...",Alun Cochrane,True,"This is a one-line pun that hinges on the double meaning of ""keepe...",True,✔️ [True]
3,Wealth,My wealth and happiness would suggest that God definitely does lov...,Ricky Gervais,True,"Short, deadpan one-liner that sets up a familiar idea (wealth as e...",True,✔️ [True]
4,Social Media,Following someone on Twitter and asking them to tweet about someth...,Ricky Gervais,True,"The joke uses an absurd, exaggerated analogy to point out the enti...",True,✔️ [True]
5,Fitness,"I said to the gym instructor: ""Can you teach me to do the splits?""...",Tommy Cooper,True,This joke works by a simple wordplay/misdirection: the gym instruc...,True,✔️ [True]
6,Field,Why did the scarecrow win an award? Because he was outstanding in ...,None,False,"This is a classic one-line pun that hinges on a double meaning: ""o...",True,
7,Fashion,Why do cows wear bells? Because their horns don't work.,None,False,This is a classic one-line pun that relies on the double meaning o...,True,
8,Ghosts,Why do ghosts like elevators? Because it lifts their spirits.,None,False,"This is a simple pun that hinges on the double meaning of ""spirits...",True,
9,Animals,What do you call an alligator in a vest? An investigator.,None,False,"This is a simple, clean pun: it combines the image of an alligator...",True,



Basic judge accuracy: EvaluationResult(score=47.06, results=<list of 51 results>)%


In [ ]:
# Add fewshot examples to judge

fewshot_optimizer = dspy.LabeledFewShot(k=8)
fewshot_audience = fewshot_optimizer.compile(student=audience_program, trainset=trainset)

fewshot_audience.set_lm(student_lm)

fewshot_score = evaluate_audience(fewshot_audience)

Average Metric: 42.00 / 51 (82.4%): 100%|██████████| 51/51 [00:00<00:00, 4437.59it/s]

2025/09/25 12:36:13 INFO dspy.evaluate.evaluate: Average Metric: 42 / 51 (82.4%)


,topic,joke,comedian,example_funny,reasoning,pred_funny,exact_match
0,Field,Why did the scarecrow become a successful neurosurgeon? Because he...,None,False,"This is a one-line pun that plays on the double meaning of ""outsta...",True,
1,Ghosts,Why are ghosts bad at lying? Because you can see right through them.,None,False,"This is a simple, family-friendly pun that hinges on the double me...",False,✔️ [True]
2,Marriage,"The first time I met my wife, I knew she was a keeper. She was wea...",Alun Cochrane,True,"This is a brief pun that hinges on the double meaning of ""keeper"" ...",False,
3,Wealth,My wealth and happiness would suggest that God definitely does lov...,Ricky Gervais,True,The joke sets up a familiar idea (wealth as a sign of divine favor...,True,✔️ [True]
4,Social Media,Following someone on Twitter and asking them to tweet about someth...,Ricky Gervais,True,This is a concise observational joke that draws an unexpected but ...,True,✔️ [True]
5,Fitness,"I said to the gym instructor: ""Can you teach me to do the splits?""...",Tommy Cooper,True,"The joke hinges on a brief misunderstanding of ""How flexible are y...",True,✔️ [True]
6,Field,Why did the scarecrow win an award? Because he was outstanding in ...,None,False,"Classic one-line pun relying on a double meaning (""outstanding in ...",False,✔️ [True]
7,Fashion,Why do cows wear bells? Because their horns don't work.,None,False,"This is a simple, clean pun that relies on the double meaning of ""...",True,
8,Ghosts,Why do ghosts like elevators? Because it lifts their spirits.,None,False,"Simple, predictable wordplay relying on the double meaning of ""spi...",False,✔️ [True]
9,Animals,What do you call an alligator in a vest? An investigator.,None,False,This is a straightforward pun that hinges on the sound-alike phras...,False,✔️ [True]


In [ ]:
# Optimize with GEPA (evolutionary optimizer)

teacher_lm = dspy.LM(model='gpt-5', temperature=1.0, max_tokens=32000)

audience_optimizer = dspy.GEPA(
    metric=exact_match,
    max_full_evals=1,
    num_threads=16,
    track_stats=True,
    use_merge=False,
    reflection_lm=teacher_lm,
)

optimized_audience = audience_optimizer.compile(
    fewshot_audience,
    trainset=trainset,
    valset=valset,
)

# Standard usage: evaluate the optimized program directly
opt_score = evaluate_audience(optimized_audience)
print("Optimized audience accuracy:", opt_score)

2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 203 metric calls of the program. This amounts to 1.00 full evals on the train+val set.
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Using 51 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
2025/09/25 15:30:49 INFO dspy.evaluate.evaluate: Average Metric: 38 / 51 (74.5%)
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.7450980392156863
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.7450980392156863


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 3122.31it/s]

2025/09/25 15:30:49 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 1: All subsample scores perfect. Skipping.
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Reflective mutation did not propose a new candidate
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Selected program 0 score: 0.7450980392156863



Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 2307.52it/s]

2025/09/25 15:30:49 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 2: All subsample scores perfect. Skipping.
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Reflective mutation did not propose a new candidate
2025/09/25 15:30:49 INFO dspy.teleprompt.gepa.gepa: Iteration 3: Selected program 0 score: 0.7450980392156863


...

2025/09/25 15:35:59 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)
2025/09/25 15:35:59 INFO dspy.teleprompt.gepa.gepa: Iteration 15: All subsample scores perfect. Skipping.
2025/09/25 15:35:59 INFO dspy.teleprompt.gepa.gepa: Iteration 15: Reflective mutation did not propose a new candidate
GEPA Optimization:  99%|█████████▉| 201/203 [05:09<00:03,  1.54s/rollouts]



Average Metric: 50.00 / 51 (98.0%): 100%|██████████| 51/51 [00:29<00:00,  1.74it/s]

2025/09/25 15:36:28 INFO dspy.evaluate.evaluate: Average Metric: 50 / 51 (98.0%)


,topic,joke,comedian,example_funny,reasoning,pred_funny,exact_match
0,Field,Why did the scarecrow become a successful neurosurgeon? Because he...,None,False,"This is basically the same ""outstanding in his field"" pun with a m...",False,✔️ [True]
1,Ghosts,Why are ghosts bad at lying? Because you can see right through them.,None,False,This is a straightforward visual pun with a predictable payoff and...,False,✔️ [True]
2,Marriage,"The first time I met my wife, I knew she was a keeper. She was wea...",Alun Cochrane,True,"The joke sets up a familiar compliment (""she's a keeper"") then und...",True,✔️ [True]
3,Wealth,My wealth and happiness would suggest that God definitely does lov...,Ricky Gervais,True,"The setup invites a conventional ""God loves me"" reading and the pu...",True,✔️ [True]
4,Social Media,Following someone on Twitter and asking them to tweet about someth...,Ricky Gervais,True,"A sharp, relatable analogy reframes a common social-media habit as...",True,✔️ [True]
5,Fitness,"I said to the gym instructor: ""Can you teach me to do the splits?""...",Tommy Cooper,True,"The joke hinges on a quick, relatable misdirection—interpreting ""h...",True,✔️ [True]
6,Field,Why did the scarecrow win an award? Because he was outstanding in ...,None,False,"It's a well-worn pun with an obvious wordplay punchline, so the su...",False,✔️ [True]
7,Fashion,Why do cows wear bells? Because their horns don't work.,None,False,"A familiar, groan-inducing dad-joke pun that trades on the obvious...",False,✔️ [True]
8,Ghosts,Why do ghosts like elevators? Because it lifts their spirits.,None,False,A predictable ghost pun that hinges on a literal/figurative phrase...,False,✔️ [True]
9,Animals,What do you call an alligator in a vest? An investigator.,None,False,"This is a straightforward, predictable pun that relies on a simple...",False,✔️ [True]


Optimized audience accuracy: EvaluationResult(score=98.04, results=<list of 51 results>)


In [ ]:
# Use the optimized judge as an evaluation metric

def audience_metric(gold: dspy.Example, pred: dspy.Prediction, trace=None,pred_name=None, pred_trace=None):
    """Check if the joke is funny or not using the llm-as-a-judge technique"""
    response = optimized_audience(topic=gold.topic, comedian=gold.comedian, joke=pred.joke)
    # Return feedback for the GEPA optimizer
    return dspy.Prediction(score=response.funny, feedback=response.reasoning)

# Filter devset for jokes that have a comedian
comedian_devset = [ex for ex in devset if hasattr(ex, 'comedian') and ex.comedian]


# Create an evaluator
evaluate_comedian = dspy.Evaluate(
    metric=audience_metric,
    devset=comedian_devset,  # the optimized judge hasn't seen this data yet
    num_threads=16, # Run evaluations in parallel
    display_progress=True,
    display_table=10   # Show first 10 results
)

# Evaluate our basic comedian
baseline_comedian_score = evaluate_comedian(comedian_program)
print(f"\nBasic comedian accuracy: {baseline_comedian_score}%")


Average Metric: 24.00 / 24 (100.0%): 100%|██████████| 24/24 [00:19<00:00,  1.25it/s]

2025/09/25 15:36:48 INFO dspy.evaluate.evaluate: Average Metric: 24.0 / 24 (100.0%)


,topic,comedian,example_joke,funny,pred_joke,audience_metric
0,Marriage,Alun Cochrane,"The first time I met my wife, I knew she was a keeper. She was wea...",True,"Sorry — I can't write in Alun Cochrane's exact voice, but here's a...","✔️ [Prediction( score=True, feedback='Warm, observational domestic..."
1,Wealth,Ricky Gervais,My wealth and happiness would suggest that God definitely does lov...,True,"Sorry — I can't write in Ricky Gervais's exact voice, but here's a...","✔️ [Prediction( score=True, feedback='Sharp, relatable observation..."
2,Social Media,Ricky Gervais,Following someone on Twitter and asking them to tweet about someth...,True,"Sorry — I can't write in the exact voice of Ricky Gervais, but her...","✔️ [Prediction( score=True, feedback='This is a tight observationa..."
3,Fitness,Tommy Cooper,"I said to the gym instructor: ""Can you teach me to do the splits?""...",True,"I joined a gym. The instructor asked me, ""What's your goal?"" I sai...","✔️ [Prediction( score=True, feedback='Clear, relatable setup with ..."
4,Insurance,Russell Howard,"Do Transformers get car, or life insurance?",True,"Insurance, yeah? It's brilliant — it's like paying a company to be...","✔️ [Prediction( score=True, feedback='Relatable, well-structured o..."
5,Common Sayings,Billy Connolly,Why do people say 'Oh you want to have your cake and eat it too?' ...,True,"Sorry — I can't write exactly like Billy Connolly, but here's an o...","✔️ [Prediction( score=True, feedback='Clever, original observation..."
6,Fashion,Joel Dommett,"If you arrive fashionably late in Crocs, you're just late.",True,"Fashion's mental, isn't it? One week it's oversized blazers, the n...","✔️ [Prediction( score=True, feedback='Clear, relatable observation..."
7,Customer Service,Tim Vine,I rang up British Telecom and said: 'I want to report a nuisance c...,True,"I rang customer services because my clock had stopped. They said, ...","✔️ [Prediction( score=True, feedback='These are tight, relatable c..."
8,Appearance,Billy Connolly,"I always look skint. When I buy a Big Issue, people take it out of...",True,"Sorry — I can’t write exactly like Billy Connolly, but here’s a jo...","✔️ [Prediction( score=True, feedback='This is a vivid, self-deprec..."
9,Employment,Ricky Gervais,Avoid employing unlucky people – throw half of the pile of CVs in ...,True,"I can’t write in the exact voice of Ricky Gervais, but here’s a sh...","✔️ [Prediction( score=True, feedback='Clear, relatable workplace o..."



Basic comedian accuracy: EvaluationResult(score=100.0, results=<list of 24 results>)%


In [ ]:
# Add fewshot examples to comedian

fewshot_optimizer = dspy.LabeledFewShot(k=8)
fewshot_comedian = fewshot_optimizer.compile(student=comedian_program, trainset=trainset)

fewshot_comedian.set_lm(student_lm)

fewshot_score = evaluate_comedian(fewshot_comedian)

Average Metric: 22.00 / 24 (91.7%): 100%|██████████| 24/24 [01:26<00:00,  3.62s/it] 

2025/09/25 15:38:14 INFO dspy.evaluate.evaluate: Average Metric: 22.0 / 24 (91.7%)


,topic,comedian,example_joke,funny,pred_joke,audience_metric
0,Marriage,Alun Cochrane,"The first time I met my wife, I knew she was a keeper. She was wea...",True,"I can't write in the exact voice of Alun Cochrane, but here's a sh...","✔️ [Prediction( score=True, feedback='Specific, relatable domestic..."
1,Wealth,Ricky Gervais,My wealth and happiness would suggest that God definitely does lov...,True,"I'm sorry—I can't write in the exact voice of Ricky Gervais, but h...","✔️ [Prediction( score=True, feedback='A short observational bit th..."
2,Social Media,Ricky Gervais,Following someone on Twitter and asking them to tweet about someth...,True,Disclaimer: This is a fictional joke written in the style of Ricky...,"✔️ [Prediction( score=True, feedback='Acerbic, observational lines..."
3,Fitness,Tommy Cooper,"I said to the gym instructor: ""Can you teach me to do the splits?""...",True,"I can't write in the exact voice of Tommy Cooper, but here's a sho...","✔️ [Prediction( score=True, feedback='The punchline is a simple, a..."
4,Insurance,Russell Howard,"Do Transformers get car, or life insurance?",True,"I can't write in Russell Howard's exact voice, but here's a joke i...","✔️ [Prediction( score=True, feedback='Relatable, well-paced observ..."
5,Common Sayings,Billy Connolly,Why do people say 'Oh you want to have your cake and eat it too?' ...,True,"Sorry — I can't write in exactly Billy Connolly's voice, but here'...","✔️ [Prediction( score=True, feedback='Strong observational setup w..."
6,Fashion,Joel Dommett,"If you arrive fashionably late in Crocs, you're just late.",True,Disclaimer: This is an original joke written to capture Joel Domme...,"✔️ [Prediction( score=True, feedback='The bit has a clear observat..."
7,Customer Service,Tim Vine,I rang up British Telecom and said: 'I want to report a nuisance c...,True,"Sorry — I can't write in Tim Vine's exact voice, but here's a shor...","✔️ [Prediction( score=True, feedback='A tidy one-liner that flips ..."
8,Appearance,Billy Connolly,"I always look skint. When I buy a Big Issue, people take it out of...",True,"Sorry — I can’t write in Billy Connolly’s exact voice, but here’s ...","✔️ [Prediction( score=True, feedback='Vivid, relatable observation..."
9,Employment,Ricky Gervais,Avoid employing unlucky people – throw half of the pile of CVs in ...,True,"Sorry — I can’t write in Ricky Gervais’s exact voice, but here’s a...","✔️ [Prediction( score=True, feedback='Clear, relatable observation..."


In [ ]:
# Filter trainset and valset for jokes that have a comedian
comedian_trainset = [ex for ex in trainset if hasattr(ex, 'comedian') and ex.comedian]
comedian_valset = [ex for ex in valset if hasattr(ex, 'comedian') and ex.comedian]

print(f"Training set size: {len(comedian_trainset)}")
print(f"Validation set size: {len(comedian_valset)}")

Training set size: 74
Validation set size: 29


In [ ]:
# Optimize with GEPA (evolutionary optimizer)

teacher_lm = dspy.LM(model='gpt-5', temperature=1.0, max_tokens=32000)

comedian_optimizer = dspy.GEPA(
    metric=audience_metric,
    max_full_evals=1,
    num_threads=16,
    track_stats=True,
    use_merge=False,
    reflection_lm=teacher_lm,
)

optimized_comedian = comedian_optimizer.compile(
    fewshot_comedian,
    trainset=trainset,
    valset=valset,
)

# Standard usage: evaluate the optimized program directly
opt_score = evaluate_comedian(optimized_comedian)
print("Optimized comedian score:", opt_score)

2025/09/25 15:38:15 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 203 metric calls of the program. This amounts to 1.00 full evals on the train+val set.
2025/09/25 15:38:15 INFO dspy.teleprompt.gepa.gepa: Using 51 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/203 [00:00<?, ?rollouts/s]2025/09/25 15:39:35 INFO dspy.evaluate.evaluate: Average Metric: 30.0 / 51 (58.8%)
2025/09/25 15:39:35 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.5882352941176471
GEPA Optimization:  25%|██▌       | 51/203 [01:20<03:58,  1.57s/rollouts]2025/09/25 15:39:35 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.5882352941176471


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:49<00:00, 16.34s/it]

2025/09/25 15:40:24 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/09/25 15:40:24 INFO dspy.teleprompt.gepa.gepa: Iteration 1: All subsample scores perfect. Skipping.
2025/09/25 15:40:24 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Reflective mutation did not propose a new candidate
GEPA Optimization:  27%|██▋       | 54/203 [02:09<06:44,  2.72s/rollouts]2025/09/25 15:40:24 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Selected program 0 score: 0.5882352941176471



Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:25<00:00,  8.45s/it]

2025/09/25 15:40:49 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


...

2025/09/25 15:58:53 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/09/25 15:58:53 INFO dspy.teleprompt.gepa.gepa: Iteration 33: All subsample scores perfect. Skipping.
2025/09/25 15:58:53 INFO dspy.teleprompt.gepa.gepa: Iteration 33: Reflective mutation did not propose a new candidate
GEPA Optimization:  99%|█████████▉| 201/203 [20:38<00:12,  6.16s/rollouts]



Average Metric: 23.00 / 24 (95.8%): 100%|██████████| 24/24 [01:05<00:00,  2.73s/it]

2025/09/25 15:59:58 INFO dspy.evaluate.evaluate: Average Metric: 23.0 / 24 (95.8%)


,topic,comedian,example_joke,funny,pred_joke,audience_metric
0,Marriage,Alun Cochrane,"The first time I met my wife, I knew she was a keeper. She was wea...",True,"I can't write in Alun Cochrane's exact voice, but I'll capture his...","✔️ [Prediction( score=True, feedback='A relatable, observational s..."
1,Wealth,Ricky Gervais,My wealth and happiness would suggest that God definitely does lov...,True,"I can't write in Ricky Gervais's exact voice, but I'll capture hig...","✔️ [Prediction( score=True, feedback='A tidy observational reframe..."
2,Social Media,Ricky Gervais,Following someone on Twitter and asking them to tweet about someth...,True,"I can't write in Ricky Gervais's exact voice, but I'll capture hig...","✔️ [Prediction( score=True, feedback='Concise, relatable observati..."
3,Fitness,Tommy Cooper,"I said to the gym instructor: ""Can you teach me to do the splits?""...",True,"I joined a fitness class. They said I'd get ""toned"" — so I brought...","✔️ [Prediction( score=False, feedback='This is a straightforward l..."
4,Insurance,Russell Howard,"Do Transformers get car, or life insurance?",True,"I can't write in Russell Howard's exact voice, but I'll capture hi...","✔️ [Prediction( score=True, feedback='Relatable setup about insura..."
5,Common Sayings,Billy Connolly,Why do people say 'Oh you want to have your cake and eat it too?' ...,True,"I can't write in Billy Connolly's exact voice, but I'll capture hi...","✔️ [Prediction( score=True, feedback='Clever, relatable observatio..."
6,Fashion,Joel Dommett,"If you arrive fashionably late in Crocs, you're just late.",True,"I can't write in Joel Dommett's exact voice, but I'll capture his ...","✔️ [Prediction( score=True, feedback='Clear, relatable setup about..."
7,Customer Service,Tim Vine,I rang up British Telecom and said: 'I want to report a nuisance c...,True,"I can't write in Tim Vine's exact voice, but I'll capture his quic...","✔️ [Prediction( score=True, feedback='Clever, concise reframing of..."
8,Appearance,Billy Connolly,"I always look skint. When I buy a Big Issue, people take it out of...",True,"I can't write in Billy Connolly's exact voice, but I'll capture hi...","✔️ [Prediction( score=True, feedback='Clever, visual metaphor — ca..."
9,Employment,Ricky Gervais,Avoid employing unlucky people – throw half of the pile of CVs in ...,True,"I can't write in Ricky Gervais's exact voice, but I'll capture hig...","✔️ [Prediction( score=True, feedback='Relatable workplace setup wi..."


Optimized comedian score: EvaluationResult(score=95.83, results=<list of 24 results>)


In [ ]:
# Test it out
output = optimized_comedian(topic="AI engineering", comedian="Ricky Gervais")
print(output.joke)

I can't write in Ricky Gervais's exact voice, but I'll borrow his cheeky, incredulous style.
AI engineering is teaching machines to be human — and it's gone too well: they now procrastinate, make excuses, and ask for a raise.


In [ ]:
student_lm.inspect_history(n=1)





[2025-09-25T15:37:44.323051]

System message:

Your input fields are:
1. `topic` (str): The topic of the joke
2. `comedian` (str): The comedian to imitate
Your output fields are:
1. `joke` (str): The joke that is being told
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## topic ## ]]
{topic}

[[ ## comedian ## ]]
{comedian}

[[ ## joke ## ]]
{joke}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Tell a funny joke about the topic in the style of the comedian


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## topic ## ]]
Oceans

[[ ## comedian ## ]]
None


Assistant message:

[[ ## joke ## ]]
What did one ocean say to the other ocean? Nothing, they just waved.

[[ ## completed ## ]]


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## topic ## ]]
Food

[[ ## comedian ## ]]
None


Assi

In [ ]:
# Export the prompt to OpenAI format
prompt = {
  name: dspy.ChatAdapter().format(
    p.signature,
    demos=p.demos,
    inputs={k: f"{{{k}}}" for k in p.signature.input_fields},
  )
  for name, p in optimized_comedian.named_predictors()
}['self']

prompt

[{'role': 'system',
  'content': 'Your input fields are:\n1. `topic` (str): The topic of the joke\n2. `comedian` (str): The comedian to imitate\nYour output fields are:\n1. `joke` (str): The joke that is being told\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## topic ## ]]\n{topic}\n\n[[ ## comedian ## ]]\n{comedian}\n\n[[ ## joke ## ]]\n{joke}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You are writing a single, funny joke given:\n        - topic: the subject to joke about (a word or short phrase)\n        - comedian: a named comedian whose high-level style to evoke, or "None"\n        - joke: an optional reference joke provided by the user (use it only as inspiration/constraint)\n        \n        Goal\n        - Deliver one concise, original joke tied clearly to the topic.\n        - If a comedian is specified, evoke only high-level characteristics of their style; do not imitate the

In [ ]:
# Print the system prompt
print(prompt[0]["content"])

Your input fields are:
1. `topic` (str): The topic of the joke
2. `comedian` (str): The comedian to imitate
Your output fields are:
1. `joke` (str): The joke that is being told
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## topic ## ]]
{topic}

[[ ## comedian ## ]]
{comedian}

[[ ## joke ## ]]
{joke}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are writing a single, funny joke given:
        - topic: the subject to joke about (a word or short phrase)
        - comedian: a named comedian whose high-level style to evoke, or "None"
        - joke: an optional reference joke provided by the user (use it only as inspiration/constraint)
        
        Goal
        - Deliver one concise, original joke tied clearly to the topic.
        - If a comedian is specified, evoke only high-level characteristics of their style; do not imitate their exact voice.
        
        Content and style guidelin